In [1]:
# setup disply parameters
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
from matplotlib.ticker import StrMethodFormatter
float_formatter = StrMethodFormatter('{x:0.03f}')
%pylab inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%config InlineBackend.figure_formats = ['retina', 'pdf', 'png']

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
pylab.rcParams['figure.figsize'] = (18, 6)

Populating the interactive namespace from numpy and matplotlib


/Users/yasser/code/projects/negmas/venv/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Develop a factory manager (agent) for the SCM world

This tutorial describes how to develop an agent for the SCM world, test it, and submit it to the ANAC 2019 SCM league.

The first step is to install negmas

```bash
pip install negmas
```
Once you have this library installed, you can start developing your factory manager:

You are told that you cannot instantiate your newly created class as did not implement the abstract (required) method ```on_negotiation_request```. That is actually the ONLY abstract method you need to implement.

In [2]:
from negmas.apps.scml import FactoryManager
class MyFactoryManager(FactoryManager):
    """My factory manager"""

As the [documentation](http://www.yasserm.com/negmas/negmas.apps.scml.html?highlight=on_negotiation_request#negmas.apps.scml.SCMLAgent.on_negotiation_request) states, this function is called whenever your factory manager receives a request from another agent to negotiate. You can either return `None` if you do not want to accept this negotiation or create  a `Negotiator` that represents your agent in it.


Your do-nothing agent is almost ready. Let's try it:

The property `stats` in `World` gives you several statistics about the world for every time-step of the simulation.

Let's check the contracts of this world:

Let's try to run a tournament with this do-nothing agent against the built-in greedy agent:

In [14]:
from negmas.apps.scml.utils import anac2019_tournament
from negmas.apps.scml import GreedyFactoryManager

n_worlds, n_steps = 4, 10 # how many worlds to run (max) for how many steps

# --------------- progress bar stuff -------------------------
from ipywidgets import IntProgress
from IPython.display import display
import time
t_progress = IntProgress(min=0, max=n_worlds) # instantiate the bar
w_progress = IntProgress(min=0, max=n_steps) # instantiate the bar
display(t_progress) # display the bar
display(w_progress) # display the bar
# --------------- progress bar stuff -------------------------

def print_progress(_, i, n) -> None:
    """Print tournament progress"""
    t_progress.value = i


def print_world_progress(world) -> None:
    """Print current world progress"""
    w_progress.value = world.current_step
    
results = anac2019_tournament(competitors=(MyFactoryManager, GreedyFactoryManager)
                              , agent_names_reveal_type=True
                              , n_runs=n_worlds, n_steps=n_steps
                              , parallelism='parallel'
                              , tournament_progress_callback=print_progress
                              , world_progress_callback=print_world_progress)

IntProgress(value=0, max=4)

IntProgress(value=0, max=10)

You can see the scores that each individual factory manager got:

In [15]:
results.scores.head()

,agent_name,agent_type,log_file,score,stats_folder,world
0,my_2_0,MyFactoryManager,None,0.000000,logs/tournaments/20190225-192632/00001/2019022...,00001/20190225-192632bunx
1,greedy_2_1,GreedyFactoryManager,None,0.031333,logs/tournaments/20190225-192632/00001/2019022...,00001/20190225-192632bunx
2,my_2_2,MyFactoryManager,None,0.000000,logs/tournaments/20190225-192632/00001/2019022...,00001/20190225-192632bunx
3,greedy_1_3,GreedyFactoryManager,None,0.042167,logs/tournaments/20190225-192632/00001/2019022...,00001/20190225-192632bunx
4,my_1_4,MyFactoryManager,None,0.000000,logs/tournaments/20190225-192632/00001/2019022...,00001/20190225-192632bunx


You can also check the total scores for each factory manager type:

In [16]:
results.total_scores

,agent_type,score
0,GreedyFactoryManager,1.505667
1,MyFactoryManager,0.000000


If you want, you can check if these differences are statistically significant using a t-test:

In [17]:
results.ttest

,a,b,p,t
0,MyFactoryManager,GreedyFactoryManager,0.004166,-3.039411


So the greedy factory manager is better than the do-nothing factory manager getting a total score of 2878 compared with only 367 for the do-nothing factory manager. Moreover, this difference is statistically significant as the p-value is 0.041 < 0.05.

Or, let's just find out who was the winner:

In [18]:
results.winners

['GreedyFactoryManager']

and what was its to score:

In [19]:
print(results.winners_scores)

[1.50566667]


Now, you completed the development of your factory manager, tested it by running it in worlds and tournaments, what exactly should you do to participate in the SCM league @ ANAC 2019:

You need to submit the following items:

- Names of all members of the team with their affiliations and email addresses
- Either a single python file with the whole implementation of your agent with any supporting code or a zip file with a single folder containing your code. In the later case, you will need to indicate the class name of your factory manager. Any factory manager names are accepted except (Insurance, Bank, MFactoryManager, CFactoryManager).
- A 2-pages academic report about your factory manager. Please check the submission website for details about this report.

That is it folks!
You can now start developing your own factory manager. Have fun.